In [ ]:
import os, shutil
import re
import time

from datetime import datetime as dt
from datetime import timedelta
from pathlib import Path
from zipfile import ZipFile

import requests
import numpy as np
import pyproj
import geopandas as gpd
import fiona
from osgeo import gdal
import rasterio as rio
from rasterio.merge import merge
from rasterio.features import shapes
from ost import Sentinel1Batch

In [ ]:
def check_product_on_asf(identifier, uname, pword):
    
    url = f"https://datapool.asf.alaska.edu/SLC/SA/{identifier}.zip"
    with requests.Session() as session:
        session.auth = (uname, pword)
        request = session.request("get", url)
        response = session.get(request.url, auth=(uname, pword), stream=True)
        return response.status_code


def create_dmp(aoi, start_date, end_date, scihub_uname, scihub_pword, asf_uname, asf_pword):
    
    # get start and end date into dt format
    event_start = dt.strptime(start_date, '%Y-%m-%d')
    event_end = dt.strptime(end_date, '%Y-%m-%d')
    
    # set a period around start and end date for data search
    search_start = dt.strftime(event_start + timedelta(days=-60), '%Y-%m-%d')
    search_end = dt.strftime(event_end + timedelta(days=30), '%Y-%m-%d')
    
    # define project dir 
    aoi_name = Path(aoi).stem
    if event_start == event_end:
        project_dir = Path().home().joinpath(f'module_results/Damage_Proxy_Maps/{aoi_name}_{start_date}')
    else:
        project_dir = Path().home().joinpath(f'module_results/Damage_Proxy_Maps/{aoi_name}_{start_date}_{end_date}')
    
    print(' Setting up project')
    s1_slc = Sentinel1Batch(
        project_dir=project_dir,
        aoi = aoi,
        start = search_start,
        end = search_end,
        product_type='SLC',
        ard_type='OST-RTC'
    )

    from ost.helpers.settings import HERBERT_USER
    
    s1_slc.scihub_uname = scihub_uname if scihub_uname else HERBERT_USER['uname'] 
    s1_slc.scihub_pword = scihub_pword if scihub_pword else HERBERT_USER['pword']
    s1_slc.asf_uname = asf_uname if asf_uname else HERBERT_USER['uname'] 
    s1_slc.asf_pword = asf_pword if asf_pword else HERBERT_USER['asf_pword']


    print(' Searching for data')
    s1_slc.search(base_url='https://scihub.copernicus.eu/dhus/')
    
    print(f' Found {len(s1_slc.inventory.relativeorbit.unique())} tracks to process')
    for i, track in enumerate(s1_slc.inventory.relativeorbit.unique()):
        
        print(f' Starting to process track {track}.')
        # filter by track
        df = s1_slc.inventory[s1_slc.inventory.relativeorbit == track].copy()
        
        # make sure all products are on ASF
        for i, row in df.iterrows():
            status = check_product_on_asf(row.identifier, s1_slc.asf_uname, s1_slc.asf_pword)
            if status != 200:
                df = df[df.identifier != row.identifier]
        
        print(df)
        break
        # get all acquisitions dates for that track
        datelist = sorted([dt.strptime(date, '%Y%m%d') for date in df.acquisitiondate.unique()])
        to_process_list = []
        for date in datelist:
            
            # cehck if we have an image after end date
            if datelist[-1] < event_end:
                print(f' No image available after the end date for track {track}', 'warning')
                break
                
            # ignore dates before start of event
            if date < event_start:
                continue
            
            # get index of date in datelist
            idx = datelist.index(date)
            if idx < 2:
                print(f' Not enough pre-event images available for track {track}', 'warning')
                break
            
            # add dates to process list, if not already there
            # we take care of the two images needed before the start date
            [to_process_list.append(date) for date in datelist[idx-2:idx+1] if date not in to_process_list]
    
            # once we added the last image after the the end of the event we can stop
            if date > event_end:
                break
        
            if len(to_process_list) < 3:
                print(f' Not enough images available for track {track}', 'warning')
                break

        # turn the dates back to strings so we can use to filter the data inventory
        final_dates = [dt.strftime(date, '%Y%m%d') for date in to_process_list]
        print(' We\'ll download/process scenes for the following dates:')
        print(final_dates)
        final_df = df[df.acquisitiondate.isin(final_dates)]
        
        print(f' {len(final_df)} identfied for download')
        print(' Downloading relevant Sentinel-1 SLC scenes ... (this may take a while)')
        s1_slc.download(final_df, mirror=2, concurrent=10, uname=s1_slc.asf_uname, pword=s1_slc.asf_pword)
            
        print(' Create burst inventory')
        s1_slc.create_burst_inventory(final_df)

        # setting ARD parameters
        print(' Setting processing parameters')
        s1_slc.ard_parameters['single_ARD']['resolution'] = 30 # in metres
        s1_slc.ard_parameters['single_ARD']['create_ls_mask'] = False
        s1_slc.ard_parameters['single_ARD']['backscatter'] = False
        s1_slc.ard_parameters['single_ARD']['coherence'] = True
        s1_slc.ard_parameters['single_ARD']['coherence_bands'] = 'VV'  # 'VV, VH'

        # production of polarimetric layers
        s1_slc.ard_parameters['single_ARD']['H-A-Alpha'] = False # does not give a lot of additional information

        # resampling of image (not so important)
        s1_slc.ard_parameters['single_ARD']['dem']['dem_name'] = "SRTM 1Sec HGT"
        s1_slc.ard_parameters['single_ARD']['dem']['image_resampling'] = 'BILINEAR_INTERPOLATION'  # 'BILINEAR_INTERPOLATION'

        # multi-temporal speckle filtering is quite effective
        s1_slc.ard_parameters['time-series_ARD']['mt_speckle_filter']['filter'] = 'Boxcar'
        s1_slc.ard_parameters['time-series_ARD']['remove_mt_speckle'] = True
        s1_slc.ard_parameters['time-scan_ARD']['metrics'] = ['min', 'max']
        s1_slc.ard_parameters['time-scan_ARD']['remove_outliers'] = False
        s1_slc.ard_parameters['mosaic']['cut_to_aoi'] = True

        # set tmp_dir
        s1_slc.config_dict['temp_dir'] = '/ram'

        #
        workers = int(4) if os.cpu_count()/4 > 4 else int(os.cpu_count()/4)
        print(f' We process {workers} bursts in parallel.')
        s1_slc.config_dict['max_workers'] = workers
        s1_slc.config_dict['executor_type'] = 'concurrent_processes'

        # process
        print(f' {len(s1_slc.burst_inventory)} bursts to process in total')
        print(" Processing... (this may take a while)")
        s1_slc.bursts_to_ards(
            timeseries=True, 
            timescan=True, 
            mosaic=False,
            overwrite=False # TO BE CHAGNED
        )
        
        
        print("calculate change and merge results")
        bursts = list(s1_slc.processing_dir.glob(f'[A,D]*{track}*'))
        
        for burst in bursts:
            print(f' Coherent change calculation for burst {burst}')
            # get track
            track_name = burst.name[:4]

            # in and out files
            coh_min = burst.joinpath('Timescan/01.coh.VV.min.tif')
            coh_max = burst.joinpath('Timescan/02.coh.VV.max.tif')
            dstnt_file = burst.joinpath(f"Timescan/ccd_{burst.name}.tif")
            print(coh_min, coh_max)
            with rio.open(coh_max) as pre_coh:
                pre_arr = pre_coh.read()
                
                # get metadata for destination file
                meta = pre_coh.meta
                meta.update(dtype='uint8', nodata=0)
            
                with rio.open(coh_min) as post_coh:
                    post_arr = post_coh.read()

                    # calulate difference
                    coh_diff = np.subtract(pre_arr, post_arr)
                    coh_diff[coh_diff < 0.27] = 0
                    coh_diff = coh_diff * 100
 
                    with rio.open(dstnt_file, 'w', **meta) as dstnt:
                        dstnt.write(coh_diff.astype('uint8'))  
            
        # -----------------------------------------
        # and merge the result
        print(' Mosaic CCD files')
        src_files_to_mosaic = []
        for file in s1_slc.processing_dir.glob(f"*[A,D]*{track}_*/Timescan/ccd*tif"):
            src = rio.open(file)
            src_files_to_mosaic.append(src)


        mosaic, out_trans = merge(src_files_to_mosaic)
        out_meta = src.profile.copy()

        # Update the metadata
        out_meta.update(
            driver='GTiff',
            height=mosaic.shape[1],
            width= mosaic.shape[2],
            transform=out_trans,
            crs=src.crs,
            tiled=True,
            blockxsize=128,
            blockysize=128,
            compress='lzw'
        )

        tmp_dir = Path(s1_slc.config_dict['temp_dir'])
        tmp_mrg = tmp_dir.joinpath(f"ccd_{track_name}.tif")
        with rio.open(tmp_mrg, "w", **out_meta) as dest:
            dest.write(mosaic)

        # close datasets
        [src.close for src in src_files_to_mosaic]

        # crop to aoi (some ost routine)
        with fiona.open(aoi, "r") as shapefile:
            shapes_ = [feature["geometry"] for feature in shapefile]

        with rio.open(tmp_mrg) as src:
            out_image, out_transform = rio.mask.mask(src, shapes_, crop=True)
            out_meta = src.profile

        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # create final output directory
        outdir = s1_slc.project_dir.joinpath('Damage_Proxy_Map')
        outdir.mkdir(parents=True, exist_ok=True)
        out_ds_tif = outdir.joinpath(f"ccd_{track_name}.tif")
        with rio.open(out_ds_tif, "w", **out_meta) as dest:
            dest.write(out_image)

        # delete tmpmerge 
        tmp_mrg.unlink()
        # -----------------------------------------

        # -----------------------------------------
        # kmz and dmp output
        # write a color file to tmp
        ctfile = tmp_dir.joinpath('colourtable.txt')
        f = open(ctfile, "w")
        ct = ["0 0 0 0 0\n"
            "27 253 246 50 255\n"
            "35 253 169 50 255\n"
            "43 253 100 50 255\n"
            "51 253 50 50 255\n"
            "59 255 10 10 255\n"
            "255 253 0 0 255"
            ]
        f.writelines(ct)
        f.close()
        
        print(' Create DPM tif')
        out_dmp_tif = outdir.joinpath(f"dmp_{track_name}.tif")
        demopts = gdal.DEMProcessingOptions(colorFilename=str(ctfile), addAlpha=True)
        gdal.DEMProcessing(str(out_dmp_tif), str(out_ds_tif), 'color-relief', options=demopts)         

        print(' Create DPM kmz')
        opts = gdal.TranslateOptions(format='KMLSUPEROVERLAY', creationOptions=["format=png"])
        gdal.Translate(str(out_dmp_tif.with_suffix('.kmz')), str(out_dmp_tif), options=opts)
        # -----------------------------------------

        # -----------------------------------------
        # polygonize (to points)
        print(' Create DPM geojson')
        with rio.open(out_ds_tif) as src:
            image = src.read()
            mask = image != 0

            results = (
                    {
                        'properties': {'raster_val': v},
                        'geometry': s
                    }
                    for i, (s, v)
                    in enumerate(
                        shapes(image, mask=mask, transform=src.transform)
                    )
            )

        geoms = list(results)  
        gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
        gpd_polygonized_raster['geometry'] = gpd_polygonized_raster['geometry'].centroid
        gpd_polygonized_raster.to_file(out_dmp_tif.with_suffix('.geojson'), driver='GeoJSON')
        # -----------------------------------------



In [ ]:
start = '2022-02-24'
end = '2022-03-14'
aoi = '/home/vollrath/DPMs/eriks/eriks_aoi.gpkg'
scihub_uname = ''
scihub_pword = ''
asf_uname = ''
asf_pword = ''

create_dmp(aoi, start, end, scihub_uname, scihub_pword, asf_uname, asf_pword)